In [1]:
from scipy.io import loadmat, savemat
import torch
import torch.nn.functional as F
import numpy as np
import time
# import yaml

import os
import matplotlib.pyplot as plt
from sdf.robot_sdf import RobotSdfCollisionNet
import tqdm


/home/dyros/.conda/envs/junheon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [3]:
print(torch.cuda.get_device_name())
device = torch.device('cuda', 0) if torch.cuda.is_available() else torch.device('cpu')
tensor_args = {'device': device, 'dtype': torch.float32}

NVIDIA GeForce RTX 3060 Laptop GPU


In [4]:
data = loadmat('../data-sampling/datasets/data_mesh.mat')['dataset']
print(data.shape)

(4950000, 19)


In [5]:
datax = np.sin(data[:,0:7])
datax = np.concatenate([datax, np.cos(data[:,0:7])], axis=1)
datax = np.concatenate([datax, data[:, 7:10]], axis=1)
print(datax.shape)

(4950000, 17)


In [6]:
L1 = 0
L2 = int(1.0 * data.shape[0])
n_size = L2
train_ratio = 0.98
test_ratio = 0.01
val_ratio = 1 - train_ratio - test_ratio
idx_train = np.arange(0, int(n_size * train_ratio))
idx_val = np.arange(idx_train[-1] + 1, int(n_size * (train_ratio + test_ratio)))
idx_test = np.arange(idx_val[-1] + 1, int(n_size))

print("Number of train data      : ", idx_train.shape[0])
print("Number of test data       : ", idx_test.shape[0])
print("Number of validation data : ", idx_val.shape[0])

Number of train data      :  4851000
Number of test data       :  49500
Number of validation data :  49500


In [7]:
# x = torch.Tensor(data[L1:L2, 0:10]).to(device, dtype=torch.float16)
x = torch.Tensor(datax[L1:L2,:]).to(device, dtype=torch.float16)
y = 100 * torch.Tensor(data[L1:L2, 10:]).to(device, dtype=torch.float16)
dof = x.shape[1]
s = 256
n_layers = 5
skips = [] # for skip-connection(s.c.)
fname = 'sdf_%dx%d_mesh.pt'%(s,n_layers)
if skips == []:
    n_layers-=1

print("Shape of Input(x)  : ", x.shape)
print("Shape of Output(y) : ", y.shape)
print("Layer Size : ", s)
print("Number of Hidden layers : ", n_layers)

Shape of Input(x)  :  torch.Size([4950000, 17])
Shape of Output(y) :  torch.Size([4950000, 9])
Layer Size :  256
Number of Hidden layers :  4


In [8]:
nn_model = RobotSdfCollisionNet(in_channels=dof, out_channels=y.shape[1], layers=[s] * n_layers, skips=skips)
nn_model.model.to(**tensor_args)
model = nn_model.model
nelem = sum([param.nelement() for param in model.parameters()])
print(repr(model))
print("Sum of parameters:%d" % nelem)

MLPRegression(
  (layers): ModuleList(
    (0): Sequential(
      (0): Sequential(
        (0): Linear(in_features=51, out_features=256, bias=True)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
      (3): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): ReLU()
      )
      (4): Sequential(
        (0): Linear(in_features=256, out_features=9, bias=True)
      )
    )
  )
)
Sum of parameters:213001


In [9]:
x_train = x[idx_train, :]
y_train = y[idx_train, :]
y_train_labels = y[idx_train, :]
y_train_labels[y_train_labels<=0] = -1
y_train_labels[y_train_labels>0] = 1

y_val_labels = y[idx_val, :]
y_val_labels[y_val_labels<=0] = -1
y_val_labels[y_val_labels>0] = 1

# scale dataset: (disabled because of nerf features!)
mean_x = torch.mean(x, dim=0) * 0.0
std_x = torch.std(x, dim=0) * 0.0 + 1.0
mean_y = torch.mean(y, dim=0) * 0.0
std_y = torch.std(y, dim=0) * 0.0 + 1.0

x_val = x[idx_val, :]
y_val = y[idx_val, :]
x_test = x[idx_test, :]
y_test = y[idx_test, :]

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5000,
                                                        threshold=0.01, threshold_mode='rel',
                                                        cooldown=0, min_lr=0, eps=1e-04, verbose=True)

In [ ]:
epochs = 100000
min_loss = 2000.0

In [ ]:
e_notsaved = 0
scaler = torch.cuda.amp.GradScaler(enabled=True) # https://computing-jhson.tistory.com/m/37
idx_close_train = (y_train[:, -1] < 10)
idx_close_val = (y_val[:, -1] < 10)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(x_train, y_train)
loader = DataLoader(train_dataset, batch_size=int(idx_train.shape[0]/5), shuffle=True, drop_last=False)

In [ ]:
for e in range(epochs):
        t0 = time.time()
        model.train()
        loss = []
        i = 0
        
        for x_batch, y_batch in loader:
            with torch.cuda.amp.autocast():
                y_pred_train = (model.forward(x_batch))
                train_loss = F.mse_loss(y_pred_train, y_batch, reduction='mean')

            scaler.scale(train_loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            loss.append(train_loss.item())

        model.eval()
        with torch.cuda.amp.autocast():
            y_pred = model.forward(x_val)
            val_loss = F.mse_loss(y_pred, y_val, reduction='mean')
            ee_loss_close = F.l1_loss(y_pred[idx_close_val][:,-1], y_val[idx_close_val][:,-1], reduction='mean')
        if e == 0:
            min_loss = val_loss
        scheduler.step(val_loss)
        train_loss = np.mean(loss)
        e_notsaved += 1
        if (val_loss < min_loss and e > 100 and e_notsaved > 100):
            e_notsaved = 0
            print('saving model', val_loss.item())
            torch.save(
                {
                    'epoch': e,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'norm': {'x': {'mean': mean_x, 'std': std_x},
                             'y': {'mean': mean_y, 'std': std_y}}
                },
                fname)
                #'sdf_convex_256_mlp_nerf_skip.pt')
            min_loss = val_loss
            print(y_pred[0, :])
            print(y_val[0, :])
            # if e > 1500:
            #     break
        print(
            "Epoch: %d (Saved at %d), Train Loss: %4.3f, Validation Loss: %4.3f (%4.3f), Epoch time: %4.3f s, LR = %4.8f" % (
                e, e-e_notsaved+1, train_loss.item(), val_loss.item(), ee_loss_close.item(), time.time() - t0,
                optimizer.param_groups[0]["lr"]))
        #print(abs(y_pred-y_val).mean(0))

In [ ]:
with torch.no_grad():
    x = x_test.float()  # [y_test[:,0] > 0.0]
    y = y_test.float()  # [y_test[:,0] > 0.0]

    y_pred = model.forward(x)
    y_pred = torch.mul(y_pred, std_y) + mean_y
    y_test = torch.mul(y, std_y) + mean_y
    print(y_test[y_test > 0.0])
    print(y_pred[y_test > 0.0])
    # print(y_pred.shape, y_test.shape)
    loss = F.l1_loss(y_pred, y_test, reduction='mean')
    print(torch.median(y_pred), torch.mean(y_pred))
    print(loss.item())